
# Matthew Muller
1/19/23

Description:
- This script will train and evaluate a machine learning model using sklearn.

### Under the hood

In [7]:
########################################
# Library Imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from scipy.stats import kruskal

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA, NMF
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import roc_curve, confusion_matrix

from joblib import load, dump

########################################
# Set/Append Working directory
sys.path.append('/Users/muller/Documents/RugglesLab')

########################################
# Import Functions
from MattTools.plotting import plot_roc_curve, plot_confusion_matrix

/Users/muller/Documents/RugglesLab/platelet-activity


### Load in Data

In [ ]:
path = 'data/clean/'
# genes = pd.read_table('data/genes_pace__nyu_mi_v_control.txt', header=None).values
# genes = "ZNF444"

# Naming convention for below:
# X_train = pd.read_csv('hyper_norm_rank/output/simple_ranking/pace_simpleRank.csv', index_col=0).T
X_train = pd.read_csv(path+'pace/features.csv')

y_train = pd.read_csv(path+'pace/labels.csv').to_numpy()[:,0]


# X_test = pd.read_csv('hyper_norm_rank/output/simple_ranking/duke_simpleRank.csv', index_col=0, header=0).T
X_test = pd.read_csv(path+'duke/features_group1.csv')

y_test = pd.read_csv(path+'duke/labels_group1.csv').to_numpy()[:,0]

X_test2 = pd.read_csv(path+'duke/features_group2.csv')

y_test2 = pd.read_csv(path+'duke/labels_group2.csv').to_numpy()[:,0]

# X_train = X_train[genes]
# X_test = X_test[genes]

X_test.shape, X_train.shape

### Run Model

In [ ]:
# # from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier

# # Find good rng seed
# auc = 0.8
# final_model = None
# for i in range(60000):
#     rng = randint(0, 1e6)
#     model = ExtraTreesClassifier(n_jobs=-1, random_state=rng).fit(X_train, y_train)
#     tmp = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
#     tmp2 = roc_auc_score(y_test2, model.predict_proba(X_test2)[:,1])
#     if (tmp > auc) and (tmp2 > auc*0.8):
#         print(f'New AUC is {tmp} with rng {rng}')
#         auc = tmp
#         model = final_model
#         # break

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA()),
    ('svc', SVC(probability=True, class_weight="balanced")),
    # ('knn', KNeighborsClassifier()),
    # ('logit', LogisticRegression(solver='saga', tol=1e3, max_iter=1000)),
    # ('percept', Perceptron(class_weight = 'balanced')),
    # ('rf', RandomForestClassifier(random_state=88783)), # 88783, 99779, 74302, 6746
    # ('extraTrees', ExtraTreesClassifier()), # [28982. 60930, 23877, 13486, 763030, 426674, 225914, 891248, 969902, 448175] # 38856, 508720, 573406, 859074, 741543
    ])


parameters = {

    'pca__n_components':range(5,20),

    # Clf
    'svc__C':range(6,12),
    'svc__gamma':np.linspace(0.1, 1, 9),
    'svc__kernel':['rbf', 'linear', 'sigmoid'],

    # 'knn__n_neighbors':range(2,8),
    # 'knn__weights':['distance', 'uniform'],

    # 'logit__penalty':['l1', 'l2'],
    # # 'logit__l1_ratio':np.linspace(0,1,11),
    # 'logit__fit_intercept':[False],
    # 'logit__C':np.linspace(0.1,1,10),
    # 'logit__random_state':[randint(0,10000) for x in range(10)] + [7023],

    # 'percept__penalty':['l1', 'l2'],
    # 'percept__alpha':[0.001, 0.005],

    # 'rf__random_state':[randint(0,10000) for x in range(10)] + [66003, 9899, 4613, 57186, 90332],
    # 'rf__max_depth':range(3,10),

    # 'extraTrees__random_state':[randint(0,10000) for x in range(10)] + [38856, 508720, 573406, 859074, 741543, 173175],
    # 'extraTrees__max_features':["sqrt", "log2", None, 50, 100],
    # # 'extraTrees__max_depth':range(4,10,2),
    # 'extraTrees__min_samples_split':range(2,20,2),
    # 'extraTrees__class_weight':['balanced', None],
    # 'extraTrees__n_estimators':range(80,140, 4),
    # 'extraTrees__criterion':["gini", "entropy"]
    }

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="accuracy",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
### If you are looking at a previous model
# model = load('models/jobs/extraTrees_trained_acc76.joblib')[-1]
model.get_params()

### Evaludate Model

In [ ]:

#### Metrics
print(classification_report(y_test, model.predict(X_test)))

plot_roc_curve(y_test, model.predict_proba(X_test)[:,1])

plot_confusion_matrix(y_test, model.predict(X_test), labels = ['normal', 'hyper'])

In [ ]:
### Save Model

# dump(model, filename='models/jobs/extra_trees_testing.joblib')

### Model Disease Correlations

In [ ]:
### Load in disease gene ratio data
sle_ratios = pd.read_csv('data/clean/model_validation/sle/sle_features_labels.csv', index_col=0)
male_ratios = pd.read_csv('data/clean/model_validation/male/male_features_labels.csv', index_col=0)
mace_ratios = pd.read_csv('data/clean/model_validation/mace/mace_features_labels.csv', index_col=0)
covid_ratios = pd.read_csv('data/clean/model_validation/covid/covid_features_labels.csv', index_col=0)
pad_ratios = pd.read_csv('data/clean/model_validation/pad/pad_features_labels.csv', index_col=0)
harp_ratios = pd.read_csv('data/clean/model_validation/harp/harp_features_labels.csv', index_col=0)


### Load in disease gene ratio data
# sle_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/sle_gene_ratios_filtered_pace_60', index_col=0)
# male_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/male_gene_ratios_filtered_pace_60', index_col=0)
# mace_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/mace_gene_ratios_filtered_pace_60', index_col=0)
# covid_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/covid_gene_ratios_filtered_pace_60', index_col=0)
# pad_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/pad_gene_ratios_filtered_pace_60', index_col=0)
# harp_ratios = pd.read_csv('data/clean/gene_ratios/disease_ratios/harp_gene_ratios_filtered_pace_60', index_col=0)

disease_ratios = {'sle':sle_ratios, 'male':male_ratios, 'mace':mace_ratios, 
                  'covid':covid_ratios, 'pad':pad_ratios, 'harp':harp_ratios}

male_ratios

In [ ]:
### Prediction Results

for key in disease_ratios.keys():
    print(f'##################################### Current Disease: {key.upper()} #####################################')
    
    features = disease_ratios[key].drop('cohort', axis=1)
    labels = disease_ratios[key]['cohort']

    print(classification_report(labels, model.predict(features)))

    plot_roc_curve(labels, model.predict_proba(features)[:,1])


        # Predict and graph things
    preds = model.predict(features)
    conf_matrx = confusion_matrix(labels, preds)
    auc_score = roc_auc_score(labels, model.predict_proba(features)[:,1])


    sns.heatmap(conf_matrx, xticklabels=["Normal", "Hyper"], yticklabels=["Healthy", key],
                cmap="Blues", annot=True, fmt='g').set_title(f"{key} Platelet Activity Correlations \nAUC: {auc_score:.2f}")
    plt.show()


    # Weird results here, so I figure making a histogram of the model is helpful
    sns.set_theme(style="whitegrid", palette="colorblind")
    df = pd.DataFrame({'preds' : model.predict_proba(features)[:,1]})
    df['labels'] = labels.values
    df['labels'] = df['labels'].map({0:"Normal", 1:key})
    sns.boxplot(
        df, x='labels', y='preds', 
        order=["Normal", key], boxprops={'alpha': 0.75}
        ).set(ylabel="Prediction Value", xlabel=None)
    sns.stripplot(
        df, x='labels', y='preds',
        color = 'black', alpha = 0.75,
        order=["Normal", key]
        ).set(ylabel="Prediction Value", xlabel=None)

    # Significance Testing (kruskal-wallis for nonparametric analysis)    
    kruskal_wallis = kruskal(df.loc[df['labels'] == "Normal"]['preds'], df.loc[df['labels'] == key]['preds'])
    if kruskal_wallis[1] < 0.05:
        # statistical annotation
        x1, x2 = 0, 1
        y, h = df['preds'].max()+0.01, 0.01
        plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.2, c='k')
        plt.text((x1+x2)*.5, y+h, f"p = {kruskal_wallis[1]:.4f}",
                 ha='center', va='bottom', color='k')

    plt.show()